# Master BigData UCM 2023

## AGRADECIMIENTOS

Este script está basado en el trabajo de Diva Flores, alumna de la clase 2 del máster. Quiero agradecerla aquí el gran trabajo que realizó para localizar los datos de población y su pre-proceso, que no es trivial, como veréis



# Visualización Avanzada
En el presente trabajo se analizarán los datos de COVID-19 que se obtienen a partir de la declaración de los casos a la Red Nacional de Vigilancia Epidemiológica (RENAVE) a través de la plataforma informática vía Web SiViES (Sistema de Vigilancia de España) que gestiona el Centro Nacional de Epidemiología (CNE). Estos datos corresponden al periodo comprendido entre el inicio de la pandemia (enero-2020) hasta el 28 de marzo de 2022.

Para realizar algunos cálculos más específicos utilizaremos datos del INE, específicamente de población por provincia y franja etaria de los últimos tres años (2020, 2021 y 2022) y los datos de polígonos que correspoden a las representaciones gráficas de las provincias de España.

**Para empezar, importamos las librerías que utilizaremos para este análisis.**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from datetime import date
from datetime import datetime


### Lectura de datasets
Realizamos la lectura de los datasets de Covid y provincias (previamente descargados) y realizamos unos primeros ajustes a los mismos.

Los archivos se encuentran en el mismo repositorio github del presente notebook.

In [ ]:
covid = pd.read_csv(r'casos_hosp_uci_def_sexo_edad_provres.csv')
prv = pd.read_csv(r'provinces_es.csv')
# geo_prv = r'SP_provincias.geojson'


**Realizamos una pre-visualización del contenido**

In [ ]:
covid.head()

In [ ]:
prv.head()

**Empezamos uniendo nuestros datasets de covid y provincias, mediante el código "provincia_iso" (en covid) o "code" en prv**

In [ ]:
cov_prv = covid.merge(prv, 
                      how='left', 
                      left_on='provincia_iso', 
                      right_on='code', 
                      suffixes=('_cov', '_prv'))

In [ ]:
cov_prv.head()

**Realizamos algunos formateos a los datos, para que nos ayuden en las visualizaciones posteriores, y eliminamos variables que no se van a usar**

In [ ]:
cov_prv['periodo'] = cov_prv['fecha'].str.slice(0, 4)
cov_prv['periodo'] = pd.to_numeric(cov_prv['periodo'])
cov_prv['fecha'] = pd.to_datetime(covid['fecha'], 
                                  format = '%Y-%m-%d', 
                                  errors = 'coerce')
cov_prv['semana'] = cov_prv['fecha'].dt.isocalendar().week

cov_prv.drop(['name', 'phone_code', 'iso2'], 
             axis = 'columns', 
             inplace=True)
cov_prv.rename(columns={'grupo_edad': 'franja_etaria'}, 
               inplace=True)
cov_prv.head()

**Realizamos la importación de los datos de población de cada provincia, correspondientes a los años 2020, 2021 y 2022.**

**Luego uniremos las tres fuentes en un único dataset.**

**IMPORTANTE: veréis que se leen directamente los datos, especificando que la coma es el separador decimal, y el punto los miles**

In [ ]:
pob_esp2020 = pd.read_csv(r'PoblacionEspana-Provincia_2020.csv',
                          sep=';',
                          encoding='latin-1'
                          ,
                          decimal = ',',
                          thousands = '.'
                         )


In [ ]:
pob_esp2020.head()

In [ ]:
pob_esp2020.describe(include = 'all')

In [ ]:
pob_esp2021 = pd.read_csv(r'PoblacionEspana-Provincia_2021.csv',
                          sep=';',
                          encoding='latin-1'
                          ,
                          decimal = ',',
                          thousands = '.'
                         )


In [ ]:
pob_esp2022 = pd.read_csv(r'PoblacionEspana-Provincia_2022.csv',
                          sep=';',
                          encoding='latin-1'
                          ,
                          decimal = ',',
                          thousands = '.'
                         )


In [ ]:
pob_esp = pd.concat([pob_esp2020, pob_esp2021, pob_esp2022])
pob_esp.head(3)

**A continuación simplemente cambiamos nombres de variables en pob_esp para facilitarnos mezclas o análisis posteriores**

In [ ]:
pob_esp.rename(columns={'Edad Simple': 'edad', 
                        'Provincias': 'provincia', 
                        'Sexo':'sexo', 
                        'Periodo':'periodo', 
                        'Total':'total'}, 
               inplace=True)

**A continuación extraemos el código postal de los dos primeros caracteres de la columna "provincia". Los pasamos  numéricos porque es el formato que tenemos en cov_prv**

In [ ]:
pob_esp['postal_code'] = pob_esp['provincia'].str.slice(0, 2)
pob_esp['postal_code'] = pd.to_numeric(pob_esp['postal_code'])

**Debemos tener los mismos códigos para hombre y mujer en pob_esp y en cov_prv**

In [ ]:
pob_esp.loc[pob_esp['sexo'] == 'Hombres', 'sexo'] = 'H'
pob_esp.loc[pob_esp['sexo'] == 'Mujeres', 'sexo'] = 'M'

**Este es quizás el punto más extraño: Para tener la edad solo con el número -p. ej. 0 años: 0- separamos toda la cadena edad, de tal manera que nos extraerá tantas columnas como palabras separadas encuentra. Encuenra 4 columnas porque la última es "100 años y más".**

**Luego se pasan los números a numeric**

In [ ]:
datos = pob_esp['edad'].str.split(expand=True)
datos.columns = ['edad2', 'edad3', 'edad4', 'edad5']
pob_esp = pd.concat([pob_esp, datos], axis=1)
pob_esp.drop(['edad3', 'edad4', 'edad5'], 
             axis = 'columns', 
             inplace=True)
pob_esp['edad2'] = pd.to_numeric(pob_esp['edad2'])
pob_esp.head(3)

**Definimos los rangos de edad igual que están en cov_prv**

In [ ]:
def franja_etaria(edad):
    if edad < 10:
        franja = '0-9'
    elif edad < 20:
        franja = '10-19'
    elif edad < 30:
        franja = '20-29'
    elif edad < 40:
        franja = '30-39'
    elif edad < 50:
        franja = '40-49'
    elif edad < 60:
        franja = '50-59'
    elif edad < 70:
        franja = '60-69'
    elif edad < 80:
        franja = '70-79'
    else:
        franja = '80+'
    return franja

In [ ]:
pob_esp['franja_etaria'] = pob_esp['edad2'].apply(franja_etaria)

**Por último dejamos nada más el año en la variable periodo**

In [ ]:
separado = pob_esp['periodo'].str.split(' ', expand=True)
pob_esp['periodo']= separado[4]
pob_esp['periodo'] = pd.to_numeric(pob_esp['periodo'])
pob_esp.head(3)

**A continuación, agrupamos los datos de poblacion por año y provincia, para cruzarlo con nuestro dataset de casos de Covid.**

In [ ]:
poblacion00 = pob_esp.groupby(['periodo', 
                             'provincia', 
                             'postal_code',
                             'franja_etaria'
                            ]).agg({'total':'sum'}).reset_index()
poblacion00.head()

In [ ]:
poblacion = poblacion00.merge(prv, 
                        how='left', 
                        left_on='postal_code', 
                        right_on='postal_code',
                        suffixes=('_cov', '_prv'))

In [ ]:
poblacion.head()

### Incidencia Acumulada a 14 días por cada 100.000 habitantes
Calculamos la IA a 14 días por cada cien mil habitantes.

In [ ]:
sum_cov = cov_prv.groupby(['fecha', 'periodo']).agg({'num_casos':'sum'})

In [ ]:
sum_cov.describe(include = 'all')

In [ ]:
def buildLaggedFeatures(s, lag=2, dropna=True):
    if type(s) is pd.DataFrame:
        new_dict={}
        for col_name in s:
            new_dict[col_name]=s[col_name]
            for l in range(1,lag+1):
                new_dict['%s_%d' %(col_name,l)]=s[col_name].shift(l)
        res=pd.DataFrame(new_dict,index=s.index)

    elif type(s) is pd.Series:
        the_range=range(lag+1)
        res=pd.concat([s.shift(i) for i in the_range],axis=1)
        res.columns=['_%d' %i for i in the_range]
    else:
        print ('Only works for DataFrame or Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res

In [ ]:
casos_trend = buildLaggedFeatures(sum_cov,lag=13,dropna=False)

In [ ]:
casos_trend = casos_trend.reset_index()

In [ ]:
casos_trend2 = pd.melt(casos_trend, 
                       id_vars=['fecha', 'periodo'], 
                       value_vars=['num_casos', 'num_casos_1', 'num_casos_2','num_casos_3', 'num_casos_4', 
                                   'num_casos_5', 'num_casos_6', 'num_casos_7', 'num_casos_8', 'num_casos_9', 
                                   'num_casos_10', 'num_casos_11', 'num_casos_12', 'num_casos_13'])

In [ ]:
casos_trend2 = casos_trend2.groupby(['fecha', 'periodo']).agg({'value':'sum'}).reset_index()
pob = poblacion.groupby(['periodo']).agg({'total':'sum'}).reset_index()
casos_trend_pob = casos_trend2.merge(pob, how='left', left_on='periodo', right_on='periodo')
casos_trend_pob['ia14d'] = casos_trend_pob['value']/casos_trend_pob['total']*100000
casos_trend_pob.head()

**Visualizamos la tendencia de la IA**

In [ ]:
from plotnine import *

In [ ]:
(
    ggplot(casos_trend_pob)  
    + aes(x = 'fecha', 
          y = 'ia14d')   
    #+ geom_rect(data=df, mapping=aes(xmin=df.x1, xmax=df.x2, ymin=0, ymax=80000, fill=df.r), color="black", alpha=0.5)
    + geom_smooth(method = "lowess", 
                  span = 0.05)

    #+ geom_hline(yintercept = 500, size = 0.5, linetype = 'dotted')
    + geom_hline(yintercept = 500, size = 0.5, linetype = 'dotted')
    + geom_vline(xintercept = '2020-03-15', size = 0.5, linetype = 'dotted')
    + geom_vline(xintercept = '2020-06-25', size = 0.5, linetype = 'dotted')
    + geom_vline(xintercept = '2020-12-10', size = 0.5, linetype = 'dotted')
    + geom_vline(xintercept = '2021-03-17', size = 0.5, linetype = 'dotted')
    + geom_vline(xintercept = '2021-06-23', size = 0.5, linetype = 'dotted')
    + geom_vline(xintercept = '2021-10-14', size = 0.5, linetype = 'dotted')
    + theme(figure_size=(8, 4),
            axis_text_x = element_text(angle = 45,
                                      hjust = 1,
                                      size = 8))
    + annotate('text', x='2020-02-10', y=600, label='Riesgo muy alto', size=9, color='red')
    + annotate('text', x='2020-05-08', y=3000, label='Primera\nola', size=9, color='blue')
    + annotate('text', x='2020-09-18', y=3000, label='Segunda\nola', size=9, color='blue')
    + annotate('text', x='2021-01-27', y=3000, label='Tercera\nola', size=9, color='blue')
    + annotate('text', x='2021-05-05', y=3000, label='Cuarta\nola', size=9, color='blue')
    + annotate('text', x='2021-08-15', y=3000, label='Quinta\nola', size=9, color='blue')
    + annotate('text', x='2022-03-10', y=3000, label='Sexta\nola', size=9, color='blue')
    + labs(title='Evolución IA a 14d por 100.000 hab.', x='Fecha', y='IA14d')
)


Se visualizan picos en determinados momentos de la pandemia, que corresponden a las diferentes olas, notándose especialmente la correspondiente a la sexta ola de la pandemia.

## Población de cada franja etaria para cada provincia

**Si anteriormente habíamos hecho el groupby de preparación de datos por fecha y periodo, ahora lo haremos por periodo y franja_etaria. Esto se debe a que queremos visualizar**

In [ ]:
cov_prv.head()

In [ ]:
cov_prv.describe(include = 'all')

In [ ]:
casos_franja_periodo = cov_prv.groupby(['periodo',
                                        'franja_etaria',
                                        'provincia_iso']).agg({'num_casos':'sum',
                                                               'num_hosp':'sum', 
                                                               'num_uci':'sum', 
                                                               'num_def':'sum'}).reset_index()

In [ ]:
casos_franja_periodo.head()

**Hacemos un merge para incorporar a nuestros datos de covid, ya agrupados, los datos de población por provincia y periodo**

In [ ]:
poblacion.head()

In [ ]:
poblacion.describe(include = 'all')

In [ ]:
casos_franja_periodo = casos_franja_periodo.merge(poblacion,
                                                  how='left', 
                                                  left_on=['periodo', 
                                                           'franja_etaria',
                                                           'provincia_iso'],
                                                  right_on=['periodo', 
                                                           'franja_etaria',
                                                           'code'])

In [ ]:
casos_franja_periodo.rename(columns={'total':'poblacion'}, inplace=True)

In [ ]:
casos_franja_periodo.head()

**Eliminamos NC y calculamos ratios de incidencia etc. por población para cada provincia y franja etaria**

In [ ]:
casos_franja_periodo = casos_franja_periodo[casos_franja_periodo['franja_etaria']!='NC']
casos_franja_periodo['casos_x_cienmil'] = casos_franja_periodo['num_casos']/casos_franja_periodo['poblacion'] * 100000
casos_franja_periodo['hosp_x_cienmil'] = casos_franja_periodo['num_hosp']/casos_franja_periodo['poblacion'] * 100000
casos_franja_periodo['uci_x_cienmil'] = casos_franja_periodo['num_uci']/casos_franja_periodo['poblacion'] * 100000
casos_franja_periodo['def_x_cienmil'] = casos_franja_periodo['num_def']/casos_franja_periodo['poblacion'] * 100000

In [ ]:
# Drop row that has all NaN values
casos_franja_periodo = casos_franja_periodo.dropna(how='all').reset_index(drop=True)



In [ ]:
casos_franja_periodo['periodo'] = casos_franja_periodo['periodo'].astype({'periodo': 'str'})

In [ ]:
casos_franja_periodo.describe(include = 'all')

## Heatmap (seaborn) de incidencia por provincia y por franja de edad

**Es importante señalar que ahora tenemos los datos ya proporcionales a población por**

**- franja etaria**
**- provincia**
**- año (2020, 21, 22)**

**y además para cada indicador: casos, hosp, uci, def**

**A continuación elegiremos un año**

In [ ]:
casos_2020 = casos_franja_periodo.loc[(casos_franja_periodo['franja_etaria'] !='NC')
                                      & (casos_franja_periodo['periodo'] == '2020')]

**Y hacemos un pivot para preparar nuestros datos eligiendo únicamente el indicador de núm. (proporcional) de casos**

In [ ]:
casos_2020_heatmap = casos_2020.pivot(index = 'name', 
                                      columns = 'franja_etaria', 
                                      values = 'casos_x_cienmil'
                                      )


In [ ]:
# Drop row that has all NaN values
casos_2020_heatmap = casos_2020_heatmap.dropna(how='all')



In [ ]:
casos_2020_heatmap.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))        
#mypalette = sns.color_palette("YlOrBr", as_cmap=True)
ax = sns.heatmap(casos_2020_heatmap
                , cmap = 'Oranges'
                #, annot=True
                , linewidth=.5
                )
ax.xaxis.tick_top()
ax.set_title('Contagios por 100000 (2020)', fontsize =20)
plt.xlabel('Franja etaria') 
plt.ylabel('Provincias') 



**Veamos las diferencias por UCI**

In [ ]:
uci_2020_heatmap = casos_2020.pivot(index = 'name', 
                                      columns = 'franja_etaria', 
                                      values = 'uci_x_cienmil'
                                      )


In [ ]:
# Drop row that has all NaN values
uci_2020_heatmap = uci_2020_heatmap.dropna(how='all')


In [ ]:
fig, ax = plt.subplots(figsize=(15,15))        
#mypalette = sns.color_palette("YlOrBr", as_cmap=True)
ax2 = sns.heatmap(uci_2020_heatmap
                , cmap = 'Oranges'
                #, annot=True
                , linewidth=.5
                )
ax2.xaxis.tick_top()
ax2.set_title('UCI por 100000 (2020)', fontsize =20)
plt.xlabel('Franja etaria') 
plt.ylabel('Provincias') 

